# T1
### Exercise 3

In this exercise we take a look at an *in-vivo* FLASH acquisition using two flip angles and a B1 (flip angle "efficiency") estimate.
How to arrive at the final linear regression, starting from the signal equation given in the last exercise is explained in detail in [**Helms et al. 2008**](https://onlinelibrary.wiley.com/doi/10.1002/mrm.21542)

In [124]:
# import packages
import numpy as np
import nibabel as nib
import pathlib as plb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mplc
import tqdm

plt.style.use('ggplot')

# set up data path
data_path = plb.Path().resolve().parents[1].absolute().joinpath("data/")
p_pdw = data_path.joinpath("PDw.nii")
p_t1w = data_path.joinpath("T1w.nii")
p_b1 = data_path.joinpath("B1map.nii")

We know the flip angles used in the acquisition (6° for pd weighting, 21° for T1w -> to see the connection to the contrast weighting refer to the last exercise plots of T1 against the flip angle alpha). The B1 map gives an estimate for how exact the flip angles are attained in the measurement.

In [103]:
# set fa
alphas = np.array([6, 21])
alphas = np.radians(alphas)
# set TR
TR = 25     # ms

# load in data
d_b1 = nib.load(p_b1)
d_t1w = nib.load(p_t1w)
d_pdw = nib.load(p_pdw)
# save affine for later
aff = d_t1w.affine
# caution B1 [%] is scaled

We could in principle use numpy and matrix multiplication to calculate the whole volume straigt away. However, from allocation considerations and the scaling of the calculations it makes sense to define it per slice
(a %%timeit for the full volume fit gave 27.8 s ± 756 ms per loop (mean ± std. dev. of 7 runs, 1 loop each, execution time)

In [104]:
# define fit function per slice (whole volume might run into allocation trouble)
def fit_slice_t1(alpha1_data_slice, alpha2_data_slice, alpha1, alpha2, b1_eff: np.ndarray = None):
    angles = [alpha1, alpha2]
    data = [alpha1_data_slice, alpha2_data_slice]
    if b1_eff is None:
        b1_eff = np.ones_like(alpha1_data_slice)
    # build design matrix
    X = np.ones((2, 2, *alpha1_data_slice.shape))
    y = np.ones((2, *alpha1_data_slice.shape))
    # assign data
    for fa_idx in range(2):
        X[fa_idx, 1] = np.divide(
            data[fa_idx], np.tan(angles[fa_idx] * b1_eff),
            where=np.tan(angles[fa_idx] * b1_eff)>np.finfo(float).eps,
            out=np.zeros_like(b1_eff)
        )
        y[fa_idx] = np.divide(
            data[fa_idx], np.sin(angles[fa_idx] * b1_eff),
            where=np.sin(angles[fa_idx] * b1_eff)>np.finfo(float).eps,
            out=np.zeros_like(b1_eff)
        )
    # move matrix axes to the end
    X = np.moveaxis(X, 0, -1)
    X = np.moveaxis(X, 0, -1)
    y = np.moveaxis(y, 0, -1)

    # find idx with enough snr
    idx_to_compute = (alpha1_data_slice > thresh) & (alpha2_data_slice > thresh) & (b1_eff > 0.3)
    # init arrays for calculation and results
    beta = np.zeros_like(alpha1_data_slice)     # for idx not in idx_compute beta stays 0
    # calculate
    # beta = (X.T X)^-1 X y
    a = np.linalg.solve(X[idx_to_compute], y[idx_to_compute])
    beta[idx_to_compute] = a[:, 1]
    return beta


In [106]:
# create array for quantitative values
q_t1 = np.zeros(d_t1w.shape)
q_t1_corr = np.zeros(d_t1w.shape)
# define signal threshold for calculating T1 (based on 800um 3T example data)
thresh = 60.0
# fit
for slice_idx in tqdm.trange(d_pdw.shape[2], desc='processing slices'):
    # load in data per slice corresponding to fa (alpha1 -> pdw, alpha2 -> t1w)
    slice_d_pdw = np.squeeze(
        np.nan_to_num(d_pdw.slicer[:,:,slice_idx:slice_idx+1].get_fdata())
    )
    slice_d_t1w = np.squeeze(
        np.nan_to_num(d_t1w.slicer[:,:,slice_idx:slice_idx+1].get_fdata())
    )
    # load in b1 data for the corrected fit - cast from %
    slice_b1 = np.squeeze(
        np.nan_to_num(d_b1.slicer[:,:,slice_idx:slice_idx+1].get_fdata())
    ) / 100.0

    # fit
    q_t1[:,:,slice_idx] = fit_slice_t1(
        slice_d_pdw, slice_d_t1w, *alphas
    )
    q_t1_corr[:,:,slice_idx] = fit_slice_t1(
        slice_d_pdw, slice_d_t1w, *alphas,
        b1_eff=slice_b1)

# transform
qt1_log = np.log(q_t1, where=q_t1>np.finfo(float).eps, out=np.zeros_like(q_t1))
qt1c_log = np.log(q_t1_corr, where=q_t1_corr>np.finfo(float).eps, out=np.zeros_like(q_t1_corr))
q_t1 = np.divide(
    1,
    np.divide(-qt1_log, TR),
    where=np.divide(-qt1_log, TR)>np.finfo(float).eps,
    out=np.zeros_like(q_t1)
)
q_t1_corr = np.divide(
    1,
    np.divide(-qt1c_log, TR),
    where=np.divide(-qt1c_log, TR)>np.finfo(float).eps,
    out=np.zeros_like(q_t1)
)

processing slices: 100%|██████████| 208/208 [00:07<00:00, 26.93it/s]


In [107]:
# save as nifti
t1_img = nib.Nifti1Image(q_t1, affine=aff)
nib.save(t1_img, data_path.joinpath("estimated_qt1.nii"))

t1_img = nib.Nifti1Image(q_t1_corr, affine=aff)
nib.save(t1_img, data_path.joinpath("estimated_qt1_b1-corrected.nii"))

Plotting - use a webviewer for nifti or MRIcroGL. In case thats not feasible, you can plot all slices here:

In [136]:
# # set axes
# dim = 1
# # arange for dimension
# plot_t1 = np.swapaxes(q_t1, dim, 0)
# num_columns = 5
# num_rows = int(np.round(plot_t1.shape[0] / num_columns))
# max_val = 4000
# # set width ratios for colorbar
# ws = np.ones(num_columns+1)
# ws[:-1] = 10
#
# fig = plt.figure(figsize=(9,1.2*num_rows))
# gs_big = fig.add_gridspec(num_rows, num_columns+1, width_ratios = ws)
#
# for plot_row in range(num_rows):
#     for plot_column in range(num_columns):
#         if plot_row*num_columns+plot_column < plot_t1.shape[0]:
#             ax = fig.add_subplot(gs_big[plot_row, plot_column])
#             ax.imshow(np.flipud(plot_t1[plot_row*num_columns+plot_column].T), interpolation='None', cmap='viridis', clim=(0,max_val))
#             ax.grid(False)
#             ax.axis(False)
# ax_cb = fig.add_subplot(gs_big[:,-1])
# cbar =fig.colorbar(cm.ScalarMappable(norm=mplc.Normalize(vmin=0, vmax=max_val), cmap='viridis'), cax=ax_cb, label=f"T1 [ms]")
# plt.tight_layout()
# plt.show()